# Athena Corporate Recruitment Interview Project 


## Craigslist Vehicles Analysis

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Uploading the data

In [ ]:
data=pd.read_csv(r"C:\Users\Ken\Desktop\My Learning\Portfolio Projects\Athena_corporate_project\craigslist_vehicles.csv")

## Data Exploration

In [ ]:
data.head(10) ##Visualize to 10 rows

In [ ]:
data.dtypes  ##Explore data types of different columns

In [ ]:
data.isnull().sum()  ##Explore Null values

In [ ]:
data.describe() ## Describe the different metrics of data before cleaning

In [ ]:
data.info() ## Show varied info about the data (range of data, shape, column names, Non_null count, datatypes) before any formating

In [ ]:
## Format different column datatypes appropriately 

data['year'] = data['year'].astype('Int64')
data['odometer'] = data['odometer'].astype('Int64')
data['posting_date'] = pd.to_datetime(data['posting_date'])
data['county'] = data['county'].astype('str')
data['state'] = data['state'].astype('str')
data['cylinders'] = pd.to_numeric(data['cylinders'], errors='coerce').astype('Int64')
data['VIN'] = pd.to_numeric(data['VIN'], errors='coerce').astype('Int64')
data['lat'] = data['lat'].astype('float')
data['long'] = data['long'].astype('float')

print(data.head(10).dtypes)


In [ ]:
data.head(10) ##Visualize top 10 rows of the data

In [ ]:
data.isnull().sum() ## Show the null values of the data after formating the datatypes 

In [ ]:
data.describe()   ## describe the different metrics of  the data after formating the datatypes 

In [ ]:
data.info()   ## Show varied info about the data (range of data, shape, column names, Non_null count, datatypes) after  formating

In [ ]:
##Understsanding the data that needs extra formating and cleaning, imputation etc

price_threshold = data['price'].quantile(0.9999)
price_outliers = data[data['price'] > data['price'].quantile(0.9999)]
max_price = data['price'].max()
min_price = data['price'].min()
count_above_threshold = price_outliers['price'].shape[0]
unrealistic_years_count = data[data['year'] <= 1900].shape[0]
missing_cylinders_count = data[data['cylinders'].isnull()].shape[0]
missing_vin = data[data['VIN'].isnull()].shape[0]
low_price_entries = data[data['price'] < 100].shape[0]
manufacturing_years_lowest_threshold = 1900
null_counts_per_row = data.isnull().sum(axis=1)
rows_with_more_than_15_nulls = data[null_counts_per_row > 15]

print(f"Rows with more than 15 nulls: {rows_with_more_than_15_nulls.shape[0]}")
print(f"Maximum Price threshold: {price_threshold}")
print(f"Minimum Price threshold: {100}")
print(f"Maximum price: {max_price}")
print(f"Minimum price: {min_price}")
print(f"manufacturing years Lowest threshold: {manufacturing_years_lowest_threshold}")
#print(f"Minimum Price threshold: {manufacturing_years_Lowest_threshold}")
print(f"Number of vehicles with price above 99th percentile: {count_above_threshold}")
print(f"Number of entries with prices below {100}: {low_price_entries}")
print(f"Number of entries with unrealistic manufacturing years: {unrealistic_years_count}")
print(f"Number of entries with missing cylinders: {missing_cylinders_count}")
print(f"Number of entries with missing VIN: {missing_vin}")


In [ ]:
price_outliers.head(40)

In [ ]:
# Count the number of rows with null and empty values
null_counts_per_row = data.isnull().sum(axis=1)
rows_with_more_than_10_nulls = data[null_counts_per_row > 15]
print(rows_with_more_than_10_nulls.shape[0])

## Data Preprocessing

In [ ]:
# Drop the unnecessary columns
data = data.drop(['id'], axis=1)
data = data.rename(columns={'Unnamed: 0': 'index'})
data = data.sort_values(by='index').reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data = data.dropna(thresh=data.shape[1] - 15)  ##Dropping the rows with more than 15 empty columns


In [ ]:
## Preprocessing price, year, cylinders, odometer, VIN, latitude and longitude columns

data = data[data['price'] <= data['price'].quantile(0.9999)]
data = data[data['year'] > 1900]
data = data.drop(columns=['cylinders'])
data['odometer'].fillna(data['odometer'].median(), inplace=True)
data = data.drop(columns=['VIN'])
data['lat'] = data.groupby('state')['lat'].transform(lambda x: x.fillna(x.mean()))
data['long'] = data.groupby('state')['long'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
data.head() ## Showing new colunms after initial preprocessing 

## Data Enrichment

In [ ]:
## Creating additional columsn from the already available columns

data = data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
data['posting_date'] = pd.to_datetime(data['posting_date'])
data['removal_date'] = pd.to_datetime(data['removal_date'])

data['time_to_sell'] = data['removal_date'] - data['posting_date']
data['posting_date_year'] = data['posting_date'].dt.year
data['posting_date_month'] = data['posting_date'].dt.month
data['posting_date_day'] = data['posting_date'].dt.day
data['removal_date_year'] = data['removal_date'].dt.year
data['removal_date_month'] = data['removal_date'].dt.month
data['removal_date_day'] = data['removal_date'].dt.day

data.head()


In [ ]:
data.dtypes ## Showing datatypes after cleaniing and preprocessing

In [ ]:
data.describe() ##Describing the data after cleaning and preprocessing

## Visualization

In [ ]:
data['posting_date'] = pd.to_datetime(data['posting_date'])
data['posting_month'] = data['posting_date'].dt.month
monthly_posted_vehicles = data.groupby('posting_month').size().reset_index(name='num_vehicles')
plt.figure(figsize=(10, 6))
sns.barplot(x='posting_month', y='num_vehicles', data=monthly_posted_vehicles)
plt.title('Number of Vehicles Posted Each Month')
plt.xlabel('Month')
plt.ylabel('Number of Vehicles Posted')
plt.show()

In [ ]:
data['posting_date'] = pd.to_datetime(data['posting_date'])
data['posting_day'] = data['posting_date'].dt.day
daily_posted_vehicles = data.groupby('posting_day').size().reset_index(name='num_vehicles')
plt.figure(figsize=(12, 6))
sns.barplot(x='posting_day', y='num_vehicles', data=daily_posted_vehicles, palette='muted')
plt.title('Number of Vehicles Posted Each Day of the Month')
plt.xlabel('Day of the Month')
plt.ylabel('Number of Vehicles Posted')
plt.show()

In [ ]:
state_listing_counts = data['state'].value_counts().reset_index().rename(columns={'index': 'state', 'state': 'num_listings'})
state_listing_counts.head(25)

In [ ]:
plt.figure(figsize=(15, 8))
sns.barplot(x='num_listings', y='count', data=state_listing_counts, palette='viridis')
plt.title('Number of Listings in Different States')
plt.xlabel('State')
plt.ylabel('Number of Listings')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.show()

In [ ]:
state_time_series = data.groupby(['state', 'posting_date']).size().reset_index(name='num_listings')
plt.figure(figsize=(15, 8))
sns.lineplot(x='posting_date', y='num_listings', hue='state', data=state_time_series, palette='viridis')
plt.title('Availability of Vehicles in Different States Over Time')
plt.xlabel('Posting Date')
plt.ylabel('Number of Listings')
plt.legend(title='State', loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
selected_region = 'ga' ##Select the state here eg (ca,wy,ny,va,ok,sc,nv,sc,nc,oh) etc

In [ ]:
data['posting_date'] = pd.to_datetime(data['posting_date'])
time_series_data = data[['posting_date', 'state', 'manufacturer']]
time_series_counts = time_series_data.groupby(['posting_date', 'state', 'manufacturer']).size().reset_index(name='num_vehicles')
filtered_data = time_series_counts[time_series_counts['state'] == selected_region]

plt.figure(figsize=(12, 6))
sns.lineplot(x='posting_date', y='num_vehicles', hue='manufacturer', data=filtered_data)
plt.title(f'Number of Available Vehicles Over Time in {selected_region}')
plt.xlabel('Posting Date')
plt.xticks(rotation=45, ha='right') 
plt.ylabel('Number of Vehicles')
plt.legend(title='manufacturer', loc='upper right', bbox_to_anchor=(1, 1)) ##supported values are 'best', 'upper right', 'upper left', 'lower left', 'lower right', 'right', 'center left', 'center right', 'lower center', 'upper center', 'center'
plt.show()


In [ ]:
selected_columns = ['price', 'year', 'posting_date', 'removal_date', 'time_to_sell', 'posting_date_month', 'posting_date_day']
selected_data = data[selected_columns]
selected_data['posting_date'] = pd.to_datetime(selected_data['posting_date'])
selected_data['removal_date'] = pd.to_datetime(selected_data['removal_date'])
correlation_matrix = selected_data.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()